In [1]:
# Parser Import
from data_source.parseData.googleFitDataParsing import *

In [2]:
processor = ParseData()
googleFit_allData = processor.allData_json(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\All Data')
googleFit_activitiesData = processor.activities_tcx(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\Activities')
user_name = "5cbdc5c5-2793-452b-a95d-0dbd3a8719ce"

start_date = '2024-10-04'
days_offset = 7
offset_sign = '-'
date_list = ['2024-09-13', '2024-09-09', '2024-09-07']
end_date = '2024-10-01'

# Add the new column to both DataFrames
googleFit_allData['userName'] = user_name
googleFit_activitiesData['userName'] = user_name

# Reorder the columns to position 'userName' as the first column
allData_columns = ['userName'] + [col for col in googleFit_allData.columns if col != 'userName']
activitiesData_columns = ['userName'] + [col for col in googleFit_activitiesData.columns if col != 'userName']

googleFit_allData = googleFit_allData[allData_columns]
googleFit_activitiesData = googleFit_activitiesData[activitiesData_columns]

In [3]:
googleFit_allData

,userName,startDate,endDate,modifiedTime,originDataSourceId,endTimeNanos,dataTypeName,startTimeNanos,rawTimestampNanos,fit_value_type,fit_value,data_source
0,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-09-16 10:18:00,2024-09-16 10:19:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462140000000000,com.google.active_minutes,1726462080000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
1,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-09-16 10:19:00,2024-09-16 10:20:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462200000000000,com.google.active_minutes,1726462140000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
2,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-09-16 10:20:00,2024-09-16 10:21:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462260000000000,com.google.active_minutes,1726462200000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
3,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-09-16 10:21:00,2024-09-16 10:22:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462320000000000,com.google.active_minutes,1726462260000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
4,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-09-16 10:22:00,2024-09-16 10:23:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462380000000000,com.google.active_minutes,1726462320000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
...,...,...,...,...,...,...,...,...,...,...,...,...
13261,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-03 08:35:39,2024-10-03 09:33:56,2024-10-03 11:02:19,,1727928236000000000,com.google.step_count.delta,1727924739000000000,0,intVal,0.0,raw:com.google.step_count.delta:com.noisefit:n...
13262,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-03 09:36:49,2024-10-03 09:49:43,2024-10-03 11:02:19,,1727929183000000000,com.google.step_count.delta,1727928409000000000,0,intVal,1280.0,raw:com.google.step_count.delta:com.noisefit:n...
13263,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-04 08:48:20,2024-10-04 09:33:47,2024-10-04 10:33:03,,1728014627000000000,com.google.step_count.delta,1728011900000000000,0,intVal,0.0,raw:com.google.step_count.delta:com.noisefit:n...
13264,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-04 09:35:47,2024-10-04 09:50:04,2024-10-04 10:33:03,,1728015604000000000,com.google.step_count.delta,1728014747000000000,0,intVal,1287.0,raw:com.google.step_count.delta:com.noisefit:n...


In [4]:
googleFit_activitiesData

,userName,Sport,Id,Lap.StartTime,Lap.DistanceMeters,Lap.TotalTimeSeconds,Lap.Calories,Lap.AverageHeartRateBpm,Lap.MaximumHeartRateBpm,Lap.Intensity,Lap.TriggerMethod,Lap.Track.Trackpoint.DistanceMeters,Lap.Track.Trackpoint.Time,HeartRateBpm
0,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:47:57.000Z,<NA>
1,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:50:00.000Z,88.0
2,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:55:00.000Z,105.0
3,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T05:00:00.000Z,109.0
4,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T05:05:00.000Z,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,0.0,2024-10-04T04:05:47.000Z,<NA>
205,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:10:00.000Z,101.0
206,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:15:00.000Z,107.0
207,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:20:00.000Z,134.0
